### Master of Applied Artificial Intelligence

**Course: TC5035 - Proyecto Integrador**

<img src="https://github.com/Medicenchapin/Proyecto-Integrador/blob/main/assets/logo.png?raw=1" alt="Image Alt Text" width="500"/>


**Feature engineering**

Tutor: Dr. Horario Martinez Alfaro


Team members:
* Ignacio Jose Aguilar Garcia - A00819762
* Alejandro Calderon Aguilar - A01795353
* Ricardo Mar Cupido - A01795394

# FE

The model contains an individualized FE:

Each row represents a client (or case).

Within each row, in the drivers column, you have something like:

```bash
[
  {"feature": "arpu_90_days", "value": 123.4, "impact": 0.05},
  {"feature": "contacts", "value": 3, "impact": -0.03},
  {"feature": "plan_postpaid", "value": 1, "impact": 0.02},
  ...
]
```

That is, each row has its own important features with different weights.
👉 That's why we can't talk about the same set of global top features at the client level,
because SHAP tells what was most relevant for that particular prediction.

## So how do we use this for the prompt?

We have two possible levels of context in the LLM prompts

🔹 **Level 1 — Global context prompt**


Includes an overview of the model and what the features represent:



*“The model uses 20 features such as ARPU, previous_calls, plan_postpaid, etc., to predict whether a prepaid user will make a purchase. The following SHAP analysis identifies the most influential features for this user.”*

👉 This gives the semantic model (Ollama or DeepSeek) the context of what the features are.

In [ ]:
import pandas as pd
import numpy as np

def build_global_context(df, feature_playbook=None, top_n=10):
    """
    df: DataFrame final que tiene columna 'drivers', donde cada row tiene
        [{"feature": str, "value": float, "impact": float}, ...]
    feature_playbook: dict opcional donde describes cada feature con lenguaje humano
                      (como el FEATURE_PLAYBOOK que ya tenías)
    """
    # Expandimos todos los drivers en un solo dataframe
    drivers_all = (
        df["drivers"]
        .explode()
        .dropna()
        .apply(pd.Series)  # -> columns: feature, value, impact
    )

    # Importancia global: mean(|impact|)
    global_importance = (
        drivers_all
        .groupby("feature")["impact"]
        .apply(lambda s: s.abs().mean())
        .sort_values(ascending=False)
        .head(top_n)
    )

    top_features = list(global_importance.index)

    # Armamos descripción de features
    lines = []
    for feat in top_features:
        if feature_playbook and feat in feature_playbook:
            desc = feature_playbook[feat]
        else:
            desc = "No description available."
        lines.append(f"- {feat}: {desc}")

    features_block = "\n".join(lines)

    global_prompt = f"""
    You are helping generate sales guidance for a prepaid telecom campaign.

    We have a machine learning model that predicts the probability that a customer will accept an offer (sale = 1).
    The model was trained on historical customer behavior and engagement indicators. 
    Higher score means the customer is more likely to buy if contacted.

    The model relies on multiple behavioral and account features. Below are the most influential features overall (averaged across customers), and what they represent:

    {features_block}

    Rules:
    - NEVER reveal internal model weights or math details.
    - NEVER invent personal/sensitive attributes not present in the features.
    - Keep tone helpful, respectful, and focused on value to customer.
    - You are allowed to explain *why the model thinks a segment is likely to buy*, in plain language.
        """.strip()

    return global_prompt


🔹 **Level 2 — Row-specific prompt (customized by customer)**


Here it uses the customer's SHAP drivers (their relevant features, values, and impacts):

In [ ]:
def build_customer_prompt(row, driver_list, extra_context_cols=None):
    """
    row: una fila de tu df (por ejemplo df.loc[idx])
         que tiene columnas humanas tipo 'state_name', 'previous_classification', etc.
         y también 'proba'.

    driver_list: lista de dicts tipo:
      [
        {"feature": "arpu_90_days", "value": 8.24, "impact": 0.32},
        ...
      ]

    extra_context_cols: lista de columnas del row que quieres pasar al LLM
                        para que hable más personalizado.
                        Ej: ["state_name", "previous_classification", "arpu_90_days", "network_age_years"]
    """

    # 1. Prepara resumen de los drivers SHAP personalizados de este cliente
    driver_lines = []
    for d in driver_list:
        driver_lines.append(
            f"- {d['feature']}: value={d['value']}, impact={d['impact']:+.3f}"
        )
    driver_block = "\n".join(driver_lines)

    # 2. Agrega contexto humano del cliente (opcional)
    context_lines = []
    if extra_context_cols:
        for col in extra_context_cols:
            if col in row:
                context_lines.append(f"{col} = {row[col]}")
    context_block = "\n".join(context_lines) if context_lines else "No additional context."

    # 3. Construye el prompt final
    prompt = f"""
    We are preparing a telemarketing/sales pitch for a prepaid mobile customer.

    Predicted probability of accepting the offer: {row['proba']:.2%}

    Relevant context for this customer:
    {context_block}

    The following factors were most influential in predicting that this customer is likely to accept an offer:
    {driver_block}

    Task:
    1. Explain, in plain language, why this customer might respond positively.
    2. Suggest how an agent should position the offer (tone, focus, what to mention).
    3. Keep it short and actionable, as guidance for a call center agent.
    4. Do NOT mention 'model', 'probability', 'algorithm', 'prediction', or 'SHAP'. Just speak as advice.
        """.strip()

    return prompt


## prompt version 1

```bash
    prompt = f"""
    The model predicts a sale probability of {proba:.2%} for this prepaid customer.
    The most influential factors for this prediction were:
    {summary}

    Explain, in natural language, why these features may lead to this prediction.
    Provide a concise and interpretable summary.
    """
```